In [1]:
#import libraries to connect SQL db
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [2]:
conn = pymysql.connect(host= 'comphcitwo.eecs.umich.edu', user = 'behavior_model_user', password = '1)wfCiyBMU', db = 'behavior_model')
cursor = conn.cursor()  #create a cursor This will allow us to execute the SQL query once we’ve written it

In [3]:
Study_ID = 179 #latest MS model study_id
run_id = 58 #
population_id = 206 #will be needed to see what are the actual states and actions of MS patients
fold_id = 57 #concerns about theta

In [4]:
Fold_State_Counts_sql = """select expected_count from Fold_State_Counts where state_id = %s
"""

In [5]:
State_Action_Policy_sql = """ select probability from State_Action_Policy where state_id = %s and action_id = %s;
"""

In [6]:
State_Transition_sql = """ select probability from State_Transition where from_state_id = %s and action_id = %s;
"""

In [7]:
to_state_sql = """ select to_state_id from State_Transition where from_state_id = %s and action_id = %s;
"""

In [8]:
get_state_code_sql = """ select state_code from State where id = %s;
"""

### Map State features to State code

In [9]:
#State features
GENDER = ["Male", "Female"] #Dem_02
AGE = ["Younger than 30", "Between 30 to 60", "60 and Older"] #Dem_01
DAYTIME_INTERVAL = ["Wake", "Morning", "Afternoon", "Evening", "Bed"] #Morning: 11am, Afternoon: 3pm, Evening: 7pm
## Lower: below 33th percentile; Normal: 33rd to 66th percentile; Higher: Above 66th percentile
PAIN = ["Lower", "Normal", "Higher", "Not Recorded"] 
FATIGUE = ["Lower", "Normal", "Higher", "Not Recorded"]
LAST_ACTIVITYBOUTS = ["Lower", "Normal", "Higher", "Not Recorded"]
EOD_PAW = ["Lower", "Normal", "Higher", "Not Recorded", None] #depends on prev action #positive affect and well-being (PAWB) higher scores indicated higher PAWB
## EOD_PAW is only recorded in the bed state, otherwise it is None.

In [10]:
states = {}
state_name_index = 1
import itertools
for x in itertools.product(GENDER, AGE, DAYTIME_INTERVAL, PAIN, FATIGUE, LAST_ACTIVITYBOUTS, EOD_PAW):
    if(not(x[2] == "Bed") and not(x[6] == None)):
        continue
    if(x[2] == "Bed" and x[6] == None):
        continue
    state = {}
    state["Gender"] = x[0]
    state["Age"] = x[1]
    state["Current Daytime Interval"] = x[2]
    state["Current Interval Pain Score"] = x[3]
    state["Current Interval Fatigue Score"] = x[4]
    state["Last Interval Activity Bouts"] = x[5]
    state["EOD Positive Affect and Well-being"] = x[6]
    state_code = 'S' + str(state_name_index)
    states[state_code] = state
    state_name_index += 1

In [11]:
len(states)

3072

In [12]:
states["S100"]

{'Gender': 'Male',
 'Age': 'Younger than 30',
 'Current Daytime Interval': 'Morning',
 'Current Interval Pain Score': 'Higher',
 'Current Interval Fatigue Score': 'Lower',
 'Last Interval Activity Bouts': 'Not Recorded',
 'EOD Positive Affect and Well-being': None}

### Map Action features to Action Code 

In [13]:
#Action 
ACTIVITYBOUTS = ["Lower", "Normal", "Higher", "Not Recorded"] #what actions have been taken in current state (this action will decide the transition to next state)
DAILY_INTERVAL_ASSESSMENT_PAIN = ["Recorded", "Not Recorded"] #what will be the pain in next state
DAILY_INTERVAL_ASSESSMENT_FATIGUE = ["Recorded", "Not Recorded"] # what will be the fatigue in next state
DAILY_INTERVAL_ASSESSMENT_PAW = ["Recorded", "Not Recorded", None] #what will be the positive affect and well-being in next state
#PAW is only recorded in the bed state, otherwise it is None. As Action decides next state, 
#DAILY_INTERVAL_ASSESSMENT_PAW action will be None in the Evening action case, which will ensure PAW = None in the bed state

In [14]:
actions = {}
action_name_index = 1
for x in itertools.product(ACTIVITYBOUTS, DAILY_INTERVAL_ASSESSMENT_PAIN, DAILY_INTERVAL_ASSESSMENT_FATIGUE, DAILY_INTERVAL_ASSESSMENT_PAW):
    action = {}
    action["Activitybouts"] = x[0]
    action["Next Interval Assessment Pain"] = x[1]
    action["Next Interval Assessment Fatigue"] = x[2]
    action["Next Interval Assessment PAW"] = x[3]
    
    action_code = 'A' + str(action_name_index)
    actions[action_code] = action
    action_name_index += 1

In [15]:
len(actions)

48

In [16]:
actions["A1"]

{'Activitybouts': 'Lower',
 'Next Interval Assessment Pain': 'Recorded',
 'Next Interval Assessment Fatigue': 'Recorded',
 'Next Interval Assessment PAW': 'Recorded'}

In [17]:
Action_list = list(actions.keys()) #All Actions 

In [18]:
#Evening action list (Actions that can be done only in the Evening states)
Evening_action_list = [key for key, items in actions.items() if items['Next Interval Assessment PAW'] != None]
Evening_action_list

['A1',
 'A2',
 'A4',
 'A5',
 'A7',
 'A8',
 'A10',
 'A11',
 'A13',
 'A14',
 'A16',
 'A17',
 'A19',
 'A20',
 'A22',
 'A23',
 'A25',
 'A26',
 'A28',
 'A29',
 'A31',
 'A32',
 'A34',
 'A35',
 'A37',
 'A38',
 'A40',
 'A41',
 'A43',
 'A44',
 'A46',
 'A47']

In [19]:
# Actions that can be done in Wake, Morning, Afternoon, Bed 
WMAB_action_list = [key for key, items in actions.items() if items['Next Interval Assessment PAW'] == None]
WMAB_action_list

['A3',
 'A6',
 'A9',
 'A12',
 'A15',
 'A18',
 'A21',
 'A24',
 'A27',
 'A30',
 'A33',
 'A36',
 'A39',
 'A42',
 'A45',
 'A48']

### All Necessary functions

In [20]:
def state_to_state_code(state):
    #input: state
    #output: state_code
    return list(states.keys())[list(states.values()).index(state)]

In [21]:
def action_to_action_code(action):
    #input: action
    #output: action_code
    return list(actions.keys())[list(actions.values()).index(action)]

### Understanding SQL Tables 

### State (id, state-code, feature_count, study_id)  where, id = state_id

In [22]:
State_sql = """ SELECT * from State WHERE study_id = 179;
"""
State_sql_df = pd.read_sql(State_sql, conn)


In [23]:
State_sql_df ## see the query at the top

,id,state_code,feature_count,study_id
0,3315187,S1,26,179
1,3315188,S2,26,179
2,3315189,S3,26,179
3,3315190,S4,26,179
4,3315191,S5,26,179
...,...,...,...,...
3067,3318254,S3068,26,179
3068,3318255,S3069,26,179
3069,3318256,S3070,26,179
3070,3318257,S3071,26,179


### Action (id, action-code, feature_count, study_id)  where, id = action_id

In [24]:
Action_sql = """select * from Action where study_id = 179;"""
Action_sql_df = pd.read_sql(Action_sql, conn)

In [25]:
Action_sql_df

,id,action_code,feature_count,study_id
0,8971,A1,10,179
1,8972,A2,10,179
2,8973,A3,10,179
3,8974,A4,10,179
4,8975,A5,10,179
5,8976,A6,10,179
6,8977,A7,10,179
7,8978,A8,10,179
8,8979,A9,10,179
9,8980,A10,10,179


### State visitation frequency (D_s)

In [26]:
%%latex
expected frequency of states $D_s$, which estimates how frequently each state occurs in behavior instances in this
model.
Table name: Fold_State_Counts

<IPython.core.display.Latex object>

In [27]:
state_vis_freq_sql = """ select * from Fold_State_Counts as FSC inner join State as S on (FSC.state_id = S.id) where S.study_id = 179; """
state_vis_freq_sql_df = pd.read_sql(state_vis_freq_sql, conn)


In [28]:
state_vis_freq_sql_df

,fold_id,state_id,expected_count,id,state_code,feature_count,study_id
0,57,3315187,4.613333e-03,3315187,S1,26,179
1,57,3315188,4.268798e-03,3315188,S2,26,179
2,57,3315189,4.510243e-03,3315189,S3,26,179
3,57,3315190,2.722751e-04,3315190,S4,26,179
4,57,3315191,7.940638e-03,3315191,S5,26,179
...,...,...,...,...,...,...,...
3067,57,3318254,1.530932e-07,3318254,S3068,26,179
3068,57,3318255,5.282775e-09,3318255,S3069,26,179
3069,57,3318256,8.213010e-09,3318256,S3070,26,179
3070,57,3318257,6.912857e-09,3318257,S3071,26,179


In [29]:
states["S5"]

{'Gender': 'Male',
 'Age': 'Younger than 30',
 'Current Daytime Interval': 'Wake',
 'Current Interval Pain Score': 'Lower',
 'Current Interval Fatigue Score': 'Normal',
 'Last Interval Activity Bouts': 'Lower',
 'EOD Positive Affect and Well-being': None}

### P(a|s) 

In [30]:
%%latex
conditional probability of actions
given situations $P(a|s)$, which estimates how likely the
person, with this behavior model, is to perform action $a$ in
state $s$ in that model.
here, state_id = from_state_id
action_id = action that has been performed in from_state

<IPython.core.display.Latex object>

In [31]:
prob_action_given_state_sql = """ select policy_id, state_id, state_code, action_id, action_code, probability from State_Action_Policy as SAP inner join State as S on (SAP.state_id = S.id) inner join Action as A on (SAP.action_id = A.id) where S.study_id = 179 and A.study_id = 179; """

In [32]:
prob_action_given_state_sql_df = pd.read_sql(prob_action_given_state_sql, conn)

In [33]:
prob_action_given_state_sql_df

,policy_id,state_id,state_code,action_id,action_code,probability
0,37,3315187,S1,8973,A3,3.400253e-01
1,37,3315187,S1,8976,A6,1.034875e-03
2,37,3315187,S1,8979,A9,2.803656e-03
3,37,3315187,S1,8982,A12,8.369997e-06
4,37,3315187,S1,8985,A15,2.695903e-01
...,...,...,...,...,...,...
55291,37,3318258,S3072,9006,A36,4.588340e-06
55292,37,3318258,S3072,9009,A39,1.243086e-02
55293,37,3318258,S3072,9012,A42,5.941540e-05
55294,37,3318258,S3072,9015,A45,6.665999e-05


### P(s'|s, a)

In [34]:
%%latex
This is called probability of going into state s', if action a is performed in state s.

The probability of the next state after the person performs
an action in the current state $P(s'|s, a)$ represents how the
actions that people perform in different situations affect
their environment and vice-versa. 

In our model there are 270336 transitions possible.                                  

<IPython.core.display.Latex object>

In [35]:
transition_proba_sql = """ select ST.from_state_id, S.state_code as from_state_code, action_id, action_code, to_state_id, FS.state_code as to_state_code, probability  from State_Transition as ST inner join State as S on (ST.from_state_id = S.id) inner join Action as A on (ST.action_id = A.id) inner join State as FS on (ST.to_state_id = FS.id) where S.study_id = 179 and A.study_id = 179 and FS.study_id = 179; """

In [36]:
transition_proba_sql_df = pd.read_sql(transition_proba_sql, conn)

In [37]:
transition_proba_sql_df

,from_state_id,from_state_code,action_id,action_code,to_state_id,to_state_code,probability
0,3315379,S193,8971,A1,3315443,S257,0.020071
1,3315379,S193,8971,A1,3315444,S258,0.012884
2,3315379,S193,8971,A1,3315445,S259,0.014971
3,3315379,S193,8971,A1,3315459,S273,0.063225
4,3315379,S193,8971,A1,3315460,S274,0.040588
...,...,...,...,...,...,...,...
270331,3318254,S3068,9018,A48,3317810,S2624,1.000000
270332,3318255,S3069,9018,A48,3317810,S2624,1.000000
270333,3318256,S3070,9018,A48,3317810,S2624,1.000000
270334,3318257,S3071,9018,A48,3317810,S2624,1.000000


### Theta 

In [38]:
%%latex
we assume a parametric reward
function that is linear in $F_{S, A}$, given unknown weight
parameters $\theta$: 
    $R(s, a) = \theta^T . F_{s_t, a_t}$

<IPython.core.display.Latex object>

In [39]:
theta_sql = """ select feature_id, feature_name, feature_index, feature_type_id, fold_id, theta  from Fold_Feature_Theta as FFT inner join Feature as F on (FFT.feature_id = F.id) where F.study_id = 179; """ 

In [40]:
theta_sql_df = pd.read_sql(theta_sql, conn)

In [41]:
#feature_type_id = {1:State_feature, 2:Action_feature} #there are 26 state feature and 10 action feature
theta_sql_df

,feature_id,feature_name,feature_index,feature_type_id,fold_id,theta
0,10634,Gender:Male,1,1,57,2.435313
1,10635,Gender:Female,2,1,57,0.679921
2,10636,Age:Younger Than 30,3,1,57,0.056990
3,10637,Age:Between 30 To 60,4,1,57,2.666766
4,10638,Age:60 And Older,5,1,57,0.056956
5,10639,Current Daytime Interval:Wake,6,1,57,0.223261
6,10640,Current Daytime Interval:Morning,7,1,57,1.482619
7,10641,Current Daytime Interval:Afternoon,8,1,57,0.066790
8,10642,Current Daytime Interval:Evening,9,1,57,1.004022
9,10643,Current Daytime Interval:Bed,10,1,57,0.445733


In [42]:
list(theta_sql_df['theta'])

[2.4353127,
 0.6799206,
 0.056989934,
 2.6667657,
 0.05695637,
 0.22326146,
 1.4826194,
 0.066789515,
 1.0040221,
 0.44573337,
 0.32523856,
 4.965209,
 0.0073088836,
 0.5265946,
 0.5027867,
 3.929241,
 0.9191457,
 0.90402764,
 0.7953072,
 1.0469936,
 2.0275648,
 0.20863518,
 0.9496609,
 0.09414342,
 0.46568388,
 0.669894,
 2.101916,
 1.9764304,
 1.0139787,
 0.550145,
 3.3115344,
 0.77819914,
 4.7721415,
 0.39413127,
 1.2493935,
 0.28817743]

### Load data for validation (7 day) (normalized using percentile method)

In [45]:
#Load the processed and normalized  dataset 
import pickle
with open('MS_normalized_df.pkl', 'rb') as f:
    data = pickle.load(f)



In [46]:
for i in range(len(data)):
    data.loc[i, 'Patient_ID'] = str(data.loc[i, 'Patient_ID'])


In [47]:
data['Gender'][::35][0:30]

0       Female
35      Female
70        Male
105     Female
140     Female
175     Female
210     Female
245       Male
280     Female
315     Female
350     Female
385     Female
420       Male
455       Male
490       Male
525     Female
560     Female
595       Male
630     Female
665     Female
700     Female
735     Female
770     Female
805     Female
840     Female
875     Female
910       Male
945     Female
980     Female
1015      Male
Name: Gender, dtype: object

In [48]:
data_p  = data[data["Patient_ID"] == "1005"]
data_p[data_p["Patient_Interval_Number"] >= 31]

,Patient_ID,Gender,Age,Daytime_Interval,Patient_Interval_Number,Pain_Score,Action_Pain,Fatigue_Score,Action_Fatigue,Last_Activitybout_State,Current_Activitybout_Action,EOD_PAW,Action_PAW
170,1005,Female,Between 30 to 60,Wake,31,Higher,Recorded,Normal,Recorded,Lower,Higher,None,None
171,1005,Female,Between 30 to 60,Morning,32,Normal,Recorded,Lower,Recorded,Higher,Higher,None,None
172,1005,Female,Between 30 to 60,Afternoon,33,Normal,Recorded,Normal,Recorded,Higher,Higher,None,None
173,1005,Female,Between 30 to 60,Evening,34,Higher,Recorded,Normal,Recorded,Higher,Higher,None,Recorded
174,1005,Female,Between 30 to 60,Bed,35,Lower,Last State,Normal,Last State,Higher,Last State,Normal,Last State


In [49]:
data.dtypes

Patient_ID                     object
Gender                         object
Age                            object
Daytime_Interval               object
Patient_Interval_Number        object
Pain_Score                     object
Action_Pain                    object
Fatigue_Score                  object
Action_Fatigue                 object
Last_Activitybout_State        object
Current_Activitybout_Action    object
EOD_PAW                        object
Action_PAW                     object
dtype: object

In [50]:
#Required Variables
people_states = {}   #people-states[Patient_No][state_no] #a dict containing lists of people, where every list contains a dictionary of all states regarding that people
people_actions = {} #a dict containing lists of people, where every list contains a dictionary of all actions regarding that people

#Patient list
Person_list = []   #total 8 ;1001-1008 patients , missing patient_id: 1017, 1021, 1051
for i in range(1001, 1111, 1):
    if(i== 1017 or i == 1021 or i == 1051):
        pass
    else:
        Person_list.append(str(i))

In [51]:
pd.options.mode.chained_assignment = None  # default='warn'
for person in Person_list:
    P = data[data['Patient_ID'] == person].reset_index()
    
    actual_states = []
    actual_actions = []
    for i in range(len(P)):
        #print(P.loc[i, 'Gender'])
        state = {}
        action = {}
        
        #Encode all actual states patients were in
        #state["Patient ID"] = person
        state["Gender"] = P.loc[i, "Gender"]
        state["Age"] = P.loc[i, "Age"]
        state["Current Daytime Interval"] = P.loc[i, "Daytime_Interval"]
        state["Current Interval Pain Score"] = P.loc[i, "Pain_Score"]
        state["Current Interval Fatigue Score"] = P.loc[i, "Fatigue_Score"]
        state["Last Interval Activity Bouts"] = P.loc[i, "Last_Activitybout_State"]
        state["EOD Positive Affect and Well-being"] = P.loc[i, "EOD_PAW"]
        actual_states.append(state)
        #Encode all actions performed by the patients
        if(P.loc[i, "Action_PAW"] == 'Last State'):
            print("Last state")
            continue
        else:
            action["Activitybouts"] = P.loc[i, "Current_Activitybout_Action"]
            action["Next Interval Assessment Pain"] = P.loc[i, "Action_Pain"]
            action["Next Interval Assessment Fatigue"] = P.loc[i, "Action_Fatigue"]
            #action["Next Interval Assessment Cognition"] = P.loc[i, "Action_Cognition"]
            #action["Next Interval Assessment Depression"] = P.loc[i, "Action_Depression"]
            #Action_Sleep_Efficiency
            #action["Next Interval Assessment Sleep Efficiency"] = P.loc[i, "Action_Sleep_Efficiency"]
            action["Next Interval Assessment PAW"] = P.loc[i, "Action_PAW"]
            actual_actions.append(action)
        
        
        
    people_states[person] = actual_states
    people_actions[person] = actual_actions
    #break
print("Finished")  

Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state
Last state

In [52]:
len(people_states["1001"])

35

In [53]:
len(people_actions["1001"])

34

In [54]:
#original states patients were in (7 days, 5 interval, 35 values for each patient): it is a dictionary where key is patient_id
people_states["1001"]

[{'Gender': 'Female',
  'Age': 'Between 30 to 60',
  'Current Daytime Interval': 'Wake',
  'Current Interval Pain Score': 'Higher',
  'Current Interval Fatigue Score': 'Normal',
  'Last Interval Activity Bouts': 'Not Recorded',
  'EOD Positive Affect and Well-being': None},
 {'Gender': 'Female',
  'Age': 'Between 30 to 60',
  'Current Daytime Interval': 'Morning',
  'Current Interval Pain Score': 'Lower',
  'Current Interval Fatigue Score': 'Lower',
  'Last Interval Activity Bouts': 'Higher',
  'EOD Positive Affect and Well-being': None},
 {'Gender': 'Female',
  'Age': 'Between 30 to 60',
  'Current Daytime Interval': 'Afternoon',
  'Current Interval Pain Score': 'Higher',
  'Current Interval Fatigue Score': 'Normal',
  'Last Interval Activity Bouts': 'Normal',
  'EOD Positive Affect and Well-being': None},
 {'Gender': 'Female',
  'Age': 'Between 30 to 60',
  'Current Daytime Interval': 'Evening',
  'Current Interval Pain Score': 'Higher',
  'Current Interval Fatigue Score': 'Higher',


In [55]:
#original actions people perform (last state has no action to have a transition so action value for each patient is one
#less than the state value)
people_actions["1001"]

[{'Activitybouts': 'Higher',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': None},
 {'Activitybouts': 'Normal',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': None},
 {'Activitybouts': 'Higher',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': None},
 {'Activitybouts': 'Lower',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': 'Recorded'},
 {'Activitybouts': 'Lower',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': None},
 {'Activitybouts': 'Higher',
  'Next Interval Assessment Pain': 'Recorded',
  'Next Interval Assessment Fatigue': 'Recorded',
  'Next Interval Assessment PAW': N

### Transform data into numpy format and save data

In [56]:
state_dict = states
action_dict = actions

In [57]:
for key, value in state_dict.items():
    print(key, value)

S1 {'Gender': 'Male', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Wake', 'Current Interval Pain Score': 'Lower', 'Current Interval Fatigue Score': 'Lower', 'Last Interval Activity Bouts': 'Lower', 'EOD Positive Affect and Well-being': None}
S2 {'Gender': 'Male', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Wake', 'Current Interval Pain Score': 'Lower', 'Current Interval Fatigue Score': 'Lower', 'Last Interval Activity Bouts': 'Normal', 'EOD Positive Affect and Well-being': None}
S3 {'Gender': 'Male', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Wake', 'Current Interval Pain Score': 'Lower', 'Current Interval Fatigue Score': 'Lower', 'Last Interval Activity Bouts': 'Higher', 'EOD Positive Affect and Well-being': None}
S4 {'Gender': 'Male', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Wake', 'Current Interval Pain Score': 'Lower', 'Current Interval Fatigue Score': 'Lower', 'Last Interval Activity Bouts': 'Not Recorded', 'EOD Positive Affect and We

S1692 {'Gender': 'Female', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Afternoon', 'Current Interval Pain Score': 'Normal', 'Current Interval Fatigue Score': 'Higher', 'Last Interval Activity Bouts': 'Not Recorded', 'EOD Positive Affect and Well-being': None}
S1693 {'Gender': 'Female', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Afternoon', 'Current Interval Pain Score': 'Normal', 'Current Interval Fatigue Score': 'Not Recorded', 'Last Interval Activity Bouts': 'Lower', 'EOD Positive Affect and Well-being': None}
S1694 {'Gender': 'Female', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Afternoon', 'Current Interval Pain Score': 'Normal', 'Current Interval Fatigue Score': 'Not Recorded', 'Last Interval Activity Bouts': 'Normal', 'EOD Positive Affect and Well-being': None}
S1695 {'Gender': 'Female', 'Age': 'Younger than 30', 'Current Daytime Interval': 'Afternoon', 'Current Interval Pain Score': 'Normal', 'Current Interval Fatigue Score': 'Not Recorded', 'La

In [58]:
# save behavior instances
behavior_instances = []
for key in people_states.keys():
    states = people_states[key]
    actions = people_actions[key]
    behavior_instance = []
    for i, state in enumerate(states):
        for key, value in state_dict.items():
            if state==value: state_code = int(key[1:])-1
        if i<len(actions):
            action = actions[i]
            for key, value in action_dict.items():
                if action==value: action_code = int(key[1:])-1
        else:
            action_code = -1
        behavior_instance.append([state_code, action_code])
    behavior_instances.append(behavior_instance)

In [59]:
np.save('behavior_instances.npy', behavior_instances)

In [110]:
# state visit frequency
state_vis_freq = []
for i in range(len(state_vis_freq_sql_df)):
    state_vis_freq.append(state_vis_freq_sql_df['expected_count'][i])

In [117]:
prob_action_given_state_sql_df

,policy_id,state_id,state_code,action_id,action_code,probability
0,37,3315187,S1,8973,A3,3.400253e-01
1,37,3315187,S1,8976,A6,1.034875e-03
2,37,3315187,S1,8979,A9,2.803656e-03
3,37,3315187,S1,8982,A12,8.369997e-06
4,37,3315187,S1,8985,A15,2.695903e-01
...,...,...,...,...,...,...
55291,37,3318258,S3072,9006,A36,4.588340e-06
55292,37,3318258,S3072,9009,A39,1.243086e-02
55293,37,3318258,S3072,9012,A42,5.941540e-05
55294,37,3318258,S3072,9015,A45,6.665999e-05


In [60]:
# probability of action given state
prob_action_given_state = np.zeros((len(state_dict), len(action_dict)))
for i in range(len(prob_action_given_state_sql_df)):
    state_code = prob_action_given_state_sql_df['state_code'][i]
    state_code = int(state_code[1:])-1
    action_code = prob_action_given_state_sql_df['action_code'][i]
    action_code = int(action_code[1:])-1
    prob_action_given_state[state_code,action_code] = prob_action_given_state_sql_df['probability'][i]
for i in range(prob_action_given_state.shape[0]):
    prob_action_given_state[i] /= prob_action_given_state[i].sum()

In [61]:
transition_proba_sql_df

,from_state_id,from_state_code,action_id,action_code,to_state_id,to_state_code,probability
0,3315379,S193,8971,A1,3315443,S257,0.020071
1,3315379,S193,8971,A1,3315444,S258,0.012884
2,3315379,S193,8971,A1,3315445,S259,0.014971
3,3315379,S193,8971,A1,3315459,S273,0.063225
4,3315379,S193,8971,A1,3315460,S274,0.040588
...,...,...,...,...,...,...,...
270331,3318254,S3068,9018,A48,3317810,S2624,1.000000
270332,3318255,S3069,9018,A48,3317810,S2624,1.000000
270333,3318256,S3070,9018,A48,3317810,S2624,1.000000
270334,3318257,S3071,9018,A48,3317810,S2624,1.000000


In [62]:
# transition probability
transition_proba = np.zeros((len(state_dict), len(action_dict), len(state_dict)))
for i in range(len(transition_proba_sql_df)):
    from_state_code = transition_proba_sql_df['from_state_code'][i]
    from_state_code = int(from_state_code[1:])-1
    action_code = transition_proba_sql_df['action_code'][i]
    action_code = int(action_code[1:])-1
    to_state_code = transition_proba_sql_df['to_state_code'][i]
    to_state_code = int(to_state_code[1:])-1
    transition_proba[from_state_code,action_code,to_state_code] = transition_proba_sql_df['probability'][i]
for i in range(transition_proba.shape[0]):
    for j in range(transition_proba.shape[1]):
        if transition_proba[i,j].sum()!=0:
            transition_proba[i,j] /= transition_proba[i,j].sum()

In [145]:
np.save('state_vis_freq', state_vis_freq)
np.save('prob_action_given_state', prob_action_given_state)
np.save('transition_proba', transition_proba)

In [155]:
state_feature_name = list(theta_sql_df['feature_name'][:26])
action_feature_name = list(theta_sql_df['feature_name'][26:])
np.save('state_feature_name', list(theta_sql_df['feature_name'][:26]))
np.save('action_feature_name', list(theta_sql_df['feature_name'][26:]))

In [181]:
state_feature = np.zeros((len(state_dict), 26))
for state_code, state_detail in state_dict.items():
    state_code = int(state_code[1:]) - 1
    for key, value in state_detail.items():
        if value is None: value = 'None'
        for i in range(len(state_feature_name)):
            if key.lower()+':'+value.lower() == state_feature_name[i].lower():
                state_feature[state_code, i] = 1

In [182]:
action_feature = np.zeros((len(action_dict), 10))
for action_code, action_detail in action_dict.items():
    action_code = int(action_code[1:]) - 1
    for key, value in action_detail.items():
        if value is None: value = 'None'
        for i in range(len(action_feature_name)):
            if key.lower()+':'+value.lower() == action_feature_name[i].lower():
                action_feature[action_code, i] = 1

In [183]:
np.save('state_feature', state_feature)
np.save('action_feature', action_feature)

In [189]:
# initial state distribution
initial_state_dist = np.zeros(len(state_dict))
for behavior_instance in behavior_instances:
    initial_state_dist[behavior_instance[0][0]] += 1
initial_state_dist /= len(behavior_instances)

In [194]:
np.save('initial_state_dist', initial_state_dist)